<a href="https://colab.research.google.com/github/Pallavi5775/text-gen/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ["WANDB_MODE"] = "disabled"

In [ ]:
!pip install datasets fuzzywuzzy evaluate sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import logging
import pandas as pd
import random
import spacy
import nltk
from nltk.corpus import wordnet
from transformers import (
    MarianMTModel,
    MarianTokenizer,
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments
)
from datasets import Dataset
from sklearn.model_selection import train_test_split
from evaluate import load
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [ ]:
# Load translation models
def load_translation_models():
    print("Loading translation models.")
    en_to_fr = {
        "tokenizer": MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr"),
        "model": MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-fr"),
    }
    fr_to_en = {
        "tokenizer": MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-fr-en"),
        "model": MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-fr-en"),
    }
    print("Translation models loaded successfully.")
    return en_to_fr, fr_to_en

en_to_fr, fr_to_en = load_translation_models()

Loading translation models.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Translation models loaded successfully.


In [ ]:
# from transformers import T5ForConditionalGeneration, T5Tokenizer
# # Initialize model and tokenizer
# model_name = "t5-small"
# tokenizer = T5Tokenizer.from_pretrained(model_name)
# model = T5ForConditionalGeneration.from_pretrained(model_name)


# from transformers import GPT2Tokenizer, GPT2LMHeadModel

# # Load GPT-2 model and tokenizer
# tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# model = GPT2LMHeadModel.from_pretrained('gpt2')

from transformers import BartTokenizer, BartForConditionalGeneration

# Load BART model and tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

# Adjust padding token if necessary
tokenizer.pad_token = tokenizer.eos_token
print("model loaded successfully.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model loaded successfully.


In [ ]:
import spacy
!pip install nltk
import nltk
nltk.download('wordnet')
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
import pandas as pd

# Load the dataset
csv_file_path = "expanded_finance_dataset_with_synthetic.csv"
print(f"Loading dataset from {csv_file_path}.")
df = pd.read_csv(csv_file_path)
print("Dataset loaded successfully.")

# Step 1: Data Cleaning
# Remove duplicates
df = df.drop_duplicates()

# Remove entries with very short `prompt` or `response` values
df['prompt'] = df['prompt'].str.lower().str.strip()
df['response'] = df['response'].str.lower().str.strip()
df = df[df['prompt'].str.len() > 5]
df = df[df['response'].str.len() > 5]

# Step 2: Display Summary
print("\nDataset Summary:")
print(f"Number of rows: {len(df)}")
print(f"Columns: {list(df.columns)}")
print("\nSample Data:")
print(df.head())

# Step 3: Class Distribution
class_distribution = df['response'].value_counts()
print("\nClass Distribution:")
print(class_distribution)

# Save the cleaned dataset for review
cleaned_file_path = "cleaned_finance_dataset.csv"
df.to_csv(cleaned_file_path, index=False)
print(f"\nCleaned dataset saved to {cleaned_file_path}.")


Loading dataset from expanded_finance_dataset_with_synthetic.csv.
Dataset loaded successfully.

Dataset Summary:
Number of rows: 48
Columns: ['prompt', 'response']

Sample Data:
                                              prompt  \
0                          how do i create a budget?   
2                          how do i create a budget?   
3   what’s the best way to plan my monthly spending?   
4           what’s the best way to plan my spending?   
5  what’s the best way to budget my monthly spend...   

                                            response  
0  start by listing your income, subtracting fixe...  
2  start by listing your income, subtracting fixe...  
3  start by listing your income, subtracting fixe...  
4  start by listing your income, subtracting fixe...  
5  start by listing your income, subtracting fixe...  

Class Distribution:
response
aim to save at least 20% of your income, but adjust based on your financial goals.                                           

In [ ]:
import random
from sklearn.utils import resample

# Synonym Replacement (using a simple dictionary for demonstration)
def synonym_replacement(sentence, synonyms):
    words = sentence.split()
    replaced_sentence = [
        random.choice(synonyms.get(word, [word])) for word in words
    ]
    return " ".join(replaced_sentence)

# Back-Translation (requires MarianMT or similar)
def back_translate(text, translator, back_translator):
    translated = translator(text, return_text=True)[0]['translation_text']
    back_translated = back_translator(translated, return_text=True)[0]['translation_text']
    return back_translated

# Generate Model Variations
def generate_variations(prompt, model, tokenizer):
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=128)
    outputs = model.generate(inputs.input_ids, max_length=128, num_return_sequences=3, do_sample=True)
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

# Upsample Rare Classes
min_class_size = 5  # Minimum number of samples per class
rare_classes = class_distribution[class_distribution < min_class_size].index
rare_df = df[df['response'].isin(rare_classes)]

if not rare_df.empty:
    upsampled_rare_df = resample(
        rare_df,
        replace=True,
        n_samples=min_class_size,
        random_state=42
    )
    df = pd.concat([df, upsampled_rare_df])

# Save Augmented Dataset
augmented_file_path = "final_augmented_finance_dataset.csv"
df.to_csv(augmented_file_path, index=False)
print(f"Augmented dataset saved to {augmented_file_path}.")


Augmented dataset saved to final_augmented_finance_dataset.csv.


In [ ]:
import pandas as pd
import re
import spacy
from nltk.corpus import wordnet
import nltk
nltk.download('wordnet')
print("Starting the data augmentation and training pipeline.")

# If GPU is available and preferred
spacy.prefer_gpu()

# Load SpaCy model for contextual synonym replacement
nlp = spacy.load("en_core_web_sm")

# Load the dataset
file_path = 'final_augmented_finance_dataset.csv'
df = pd.read_csv(file_path)


# Function to clean text
def clean_text(text):
    """
    Clean text by removing extra spaces, correcting typos, and normalizing cases.
    """
    # Remove extra whitespace and normalize casing
    text = text.strip().lower()
    # Remove special characters and normalize
    text = re.sub(r"[^a-zA-Z0-9\s']", "", text)
    return text

# Function to replace synonyms
def replace_with_synonyms(sentence):
    """
    Replace words with synonyms for minor diversification.
    """
    doc = nlp(sentence)
    new_sentence = []
    for token in doc:
        if token.pos_ in ["NOUN", "VERB", "ADJ"]:  # Only replace key words
            synonyms = wordnet.synsets(token.text)
            if synonyms:
                synonym = synonyms[0].lemmas()[0].name()  # Use first synonym
                new_sentence.append(synonym)
            else:
                new_sentence.append(token.text)
        else:
            new_sentence.append(token.text)
    return " ".join(new_sentence)

# Clean the dataset
def clean_dataset(df):
    """
    Clean and preprocess the dataset.
    """
    # Step 1: Remove duplicates
    df.drop_duplicates(subset=["prompt", "response"], inplace=True)

    # Step 2: Clean prompts and responses
    df['prompt'] = df['prompt'].apply(clean_text)
    df['response'] = df['response'].apply(clean_text)

    # Step 3: Drop exact duplicates again post-cleaning
    df.drop_duplicates(subset=["prompt", "response"], inplace=True)

    # Step 4: Replace synonyms for diversification (optional)
    df['prompt'] = df['prompt'].apply(replace_with_synonyms)

    return df

# Apply cleaning
cleaned_df = clean_dataset(df)

# Save the cleaned dataset
output_file_path = 'cleaned_final_finance_dataset.csv'
cleaned_df.to_csv(output_file_path, index=False)

print(f"Cleaned dataset saved to {output_file_path}")


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Starting the data augmentation and training pipeline.
Cleaned dataset saved to cleaned_final_finance_dataset.csv


In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re

# Load dataset
file_path = 'cleaned_final_finance_dataset.csv'
df = pd.read_csv(file_path)

# Remove exact duplicates
df = df.drop_duplicates(subset=['prompt', 'response'])

# Function to clean text
def clean_text(text):
    text = text.lower().strip()  # Convert to lowercase and strip extra spaces
    text = re.sub(r"[^a-z0-9\s']", "", text)  # Remove special characters
    text = re.sub(r"\s+", " ", text)  # Replace multiple spaces with a single space
    return text

# Apply cleaning to both columns
df['prompt'] = df['prompt'].apply(clean_text)
df['response'] = df['response'].apply(clean_text)

# Function to standardize near-duplicate prompts
def standardize_prompts(df):
    unique_prompts = {}
    for prompt in df['prompt'].unique():
        for key in unique_prompts.keys():
            if fuzz.ratio(prompt, key) > 85:  # Adjust similarity threshold as needed
                unique_prompts[prompt] = key
                break
        else:
            unique_prompts[prompt] = prompt
    df['prompt'] = df['prompt'].map(unique_prompts)
    return df

df = standardize_prompts(df)

# Remove duplicate prompts (keeping the first occurrence)
df = df.drop_duplicates(subset=['prompt'])

# Save cleaned dataset
output_path = 'cleaned_final_finance_dataset23.csv'
df.to_csv(output_path, index=False)

print(f"Cleaned dataset saved to {output_path}")


Cleaned dataset saved to cleaned_final_finance_dataset23.csv


In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz

# Load the dataset
file_path = "cleaned_final_finance_dataset23.csv"
df = pd.read_csv(file_path)

# Function to clean and standardize text
def clean_text(text):
    text = text.lower().strip()  # Lowercase and remove extra spaces
    text = text.replace("taxwriteoff", "tax write-off")
    text = text.replace("runlow", "allocate")
    text = text.replace("pickings", "considerations")
    text = text.capitalize()  # Add capitalization
    return text

# Apply cleaning function
df['prompt'] = df['prompt'].apply(clean_text)
df['response'] = df['response'].apply(clean_text)

# Standardize similar prompts
def standardize_prompts(df, threshold=85):
    unique_prompts = {}
    for prompt in df['prompt'].unique():
        for key in unique_prompts.keys():
            if fuzz.ratio(prompt, key) > threshold:
                unique_prompts[prompt] = key
                break
        else:
            unique_prompts[prompt] = prompt
    df['prompt'] = df['prompt'].map(unique_prompts)
    return df

df = standardize_prompts(df)

# Remove duplicates
df = df.drop_duplicates(subset=['prompt', 'response'])

# Save the cleaned dataset
output_path = "cleaned_final_finance_dataset232.csv"
df.to_csv(output_path, index=False)

print(f"Cleaned dataset saved to {output_path}")


Cleaned dataset saved to cleaned_final_finance_dataset232.csv


In [ ]:
!apt-get update && apt-get install -y locales
!locale-gen en_US.UTF-8
!update-locale LC_ALL=en_US.UTF-8 LANG=en_US.UTF-8


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,566 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,448 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packa

In [ ]:
!pip uninstall nltk -y
!pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 15.8 MB/s eta 0:00:00


In [ ]:
import nltk

#Download the required tagger and ensure proper resource exists
nltk.download('averaged_perceptron_tagger')





[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
nltk.download('averaged_perceptron_tagger_eng')  # Force attempt for the expected resource

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...


FileNotFoundError: [Errno 2] No such file or directory: '/root/nltk_data/taggers/averaged_perceptron_tagger_eng/averaged_perceptron_tagger.weights.json'

In [ ]:
import os
from nltk.data import find

# Locate the existing 'averaged_perceptron_tagger' directory
source_path = find('taggers/averaged_perceptron_tagger/').path
target_dir = os.path.join(source_path.rsplit('/', 1)[0], 'averaged_perceptron_tagger_eng')

# Create the directory if it doesn't exist
os.makedirs(target_dir, exist_ok=True)

# Copy or symlink the required files
for filename in ['averaged_perceptron_tagger.pickle', 'averaged_perceptron_tagger.weights.json']:
    source_file = os.path.join(source_path, filename)
    target_file = os.path.join(target_dir, filename)
    if not os.path.exists(target_file):
        os.symlink(source_file, target_file)



FileExistsError: [Errno 17] File exists: '/root/nltk_data/taggers/averaged_perceptron_tagger/averaged_perceptron_tagger.weights.json' -> '/root/nltk_data/taggers/averaged_perceptron_tagger_eng/averaged_perceptron_tagger.weights.json'

In [ ]:
import nltk
from nltk.tag.perceptron import PerceptronTagger

# Override the tagger to default to the existing English model
original_load = PerceptronTagger.load_from_json

def patched_load_from_json(self, lang="eng"):
    try:
        return original_load(self, lang)
    except FileNotFoundError:
        return original_load(self, "en")  # Use the default English model instead

PerceptronTagger.load_from_json = patched_load_from_json



In [ ]:
!pip install --upgrade nltk nlpaug


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 27.0 MB/s eta 0:00:00


In [ ]:
from nlpaug.augmenter.word import SynonymAug

# Define the augmenter with fine-tuned parameters
aug = SynonymAug(
    aug_src='wordnet',
    stopwords=["do", "I", "a", "budget"],  # Prevent replacing specific words
    aug_min=1,  # Minimum number of words to augment
    aug_max=1  # Maximum number of words to augment
)

# Test input
sample_text = "How do I make a budget?"

# Apply augmentation
augmented_text = aug.augment(sample_text)
print(augmented_text)







['How do I make a budget?']


In [ ]:
import pandas as pd
from nlpaug.augmenter.word import SynonymAug
from collections import Counter
import nltk
from nltk.corpus import stopwords

# Ensure NLTK stopwords are downloaded
nltk.download('stopwords')

# Load your dataset
df = pd.read_csv('cleaned_final_finance_dataset232.csv')  # Adjust the file name/path

# Combine all text data (both prompt and response if applicable)
text_data = ' '.join(df['prompt'].tolist())

# Tokenize and count word frequencies
words = text_data.split()
word_freq = Counter(words)

# Get common English stopwords
common_stopwords = set(stopwords.words('english'))

# Identify dynamic stopwords: common dataset-specific words or frequent ones
dynamic_stopwords = {word for word, count in word_freq.items() if count > 10 or word.lower() in common_stopwords}

# Log the dynamic stopwords for review
print("Dynamic Stopwords:", dynamic_stopwords)

# Initialize the augmenter with dynamically identified stopwords
aug = SynonymAug(aug_src='wordnet', stopwords=list(dynamic_stopwords), aug_min=1, aug_max=2)

# Function to augment and ensure a change occurs
def augment_text(text):
    augmented = aug.augment(text)
    # If no augmentation occurred, return the original text
    return augmented if augmented != text else f"Unchanged: {text}"

# Apply synonym augmentation to the 'prompt' column
df['prompt'] = df['prompt'].apply(augment_text)

# Check the augmented data
print(df[['prompt', 'response']].head())

# # Save the augmented dataset for later use
df.to_csv('augmented_dataset.csv', index=False)




Dynamic Stopwords: {'a', 'of', 'how', 'and', 'do', 'What', 'when', 'is', 'me', 'them', 'there', 'you', 'does', 'between', 'on', 'it', 'with', 'i', 'are', 'why', 'Of', 'How', 'to', 'my', 'can', 'Is', 'for', 'should', 'Can', 'the'}
                                              prompt  \
0                        [How do i prepare a budget]   
1  [What endorse the best manner to plan my month...   
2            [Can you assist me organize my expense]   
3                      [Aid me form my disbursement]   
4             [How much of my income should i write]   

                                            response  
0  Start by listing your income subtracting fixed...  
1  Start by listing your income subtracting fixed...  
2  Start by listing your income subtracting fixed...  
3  Start by listing your income subtracting fixed...  
4  Aim to save at least 20 of your income but adj...  


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import pandas as pd
from nlpaug.augmenter.word import SynonymAug
import ast

# Load dataset
df = pd.read_csv("augmented_dataset.csv")

# Initialize the augmenter
aug = SynonymAug(aug_src="wordnet", stopwords=["how", "do", "i", "my", "is", "the", "to", "and", "budget"])

# Function to apply augmentation to the prompt in a list format
def augment_prompt(prompt):
    # Convert string representation of list to actual list
    prompt_list = ast.literal_eval(prompt)
    augmented_list = []

    for item in prompt_list:
        augmented_text = aug.augment(item)  # Apply augmentation
        augmented_list.append(augmented_text)

    # Convert list back to a string without brackets and quotes
    return augmented_list[0][0]  # Assuming only one prompt per list

# Apply augmentation to the 'prompt' column
df["prompt"] = df["prompt"].apply(augment_prompt)

# Save augmented dataset
df.to_csv("synonym_augmented_dataset.csv", index=False)
print("Synonym augmented dataset saved as synonym_augmented_dataset.csv.")




Synonym augmented dataset saved as synonym_augmented_dataset.csv.


In [ ]:
from transformers import MarianMTModel, MarianTokenizer
import pandas as pd

def back_translate(text, src_lang="en", tgt_lang="fr"):
    # Initialize the translation models and tokenizers
    model_name = f"Helsinki-NLP/opus-mt-{src_lang}-{tgt_lang}"
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)

    # Translate to target language
    tgt_text = model.generate(**tokenizer(text, return_tensors="pt", padding=True))
    tgt_text_decoded = tokenizer.decode(tgt_text[0], skip_special_tokens=True)

    # Translate back to source language
    model_name_back = f"Helsinki-NLP/opus-mt-{tgt_lang}-{src_lang}"
    tokenizer_back = MarianTokenizer.from_pretrained(model_name_back)
    model_back = MarianMTModel.from_pretrained(model_name_back)

    src_text = model_back.generate(**tokenizer_back(tgt_text_decoded, return_tensors="pt", padding=True))
    src_text_decoded = tokenizer_back.decode(src_text[0], skip_special_tokens=True)


    return src_text_decoded  # Return the string directly

# Load dataset
df = pd.read_csv("augmented_dataset.csv")
df["prompt"] = df["prompt"].apply(lambda x: ' '.join(eval(x)) if isinstance(eval(x), list) else x)
print(df["prompt"])
# Apply back translation to the 'prompt' column
df["prompt"] = df["prompt"].apply(lambda x: back_translate(x))

# Save the augmented dataset
df.to_csv("back_translation_augmented_dataset.csv", index=False)
print("Back translation augmented dataset saved as back_translation_augmented_dataset.csv.")





0                             How do i prepare a budget
1     What endorse the best manner to plan my monthl...
2                 Can you assist me organize my expense
3                           Aid me form my disbursement
4                  How much of my income should i write
5                        How much should i pull through
6     How much of my income should i save for retire...
7            What portion of income should go to rescue
8              How much of income should go to nest egg
9          Is there a delivery linguistic rule of thumb
10    Is there a savings rule is there a rescue formula
11                   How much should i save for retreat
12    Of my income should kick the bucket to my reti...
13     What second the best fashion to save for retreat
14                           How do i salve for retreat
15                   What are taxation tax write - sour
16        What are tax indite - bump off for income tax
17        Can you explicate revenue enhancement 

In [ ]:

import pandas as pd
def paraphrase(text):
    # Ensure input text is treated as a string and remove unwanted characters
    text = str(text).strip()  # Remove leading and trailing spaces

    # Apply paraphrasing only if text is not 'False' or empty
    if text and text.lower() != "false":
        # Format the text for paraphrasing using the T5 model
        input_text = f"paraphrase: {text} </s>"
        encoding = tokenizer(input_text, return_tensors="pt", padding=True)
        outputs = model.generate(**encoding, max_length=50, num_beams=5, num_return_sequences=1, early_stopping=True)

        paraphrased_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        paraphrased_text = paraphrased_text.replace("Paraphrase:", "").strip()
       # Clean the paraphrased text
        paraphrased_text = clean_paraphrased_text(paraphrased_text)
        print(paraphrased_text)
        return paraphrased_text
    else:
        return ''  # Return the original text if it's invalid or empty
def clean_paraphrased_text(text):
    # Remove unwanted "Paraphrase" or special characters from the paraphrased text
    text = text.replace("Paraphrase", "").strip()  # Remove any "Paraphrase" prefix
    text = text.replace("paraphrase:", "").strip()  # Remove any "paraphrase:" prefix
    text = text.replace("[", "").replace("]", "")  # Remove unwanted square brackets
    text = text.replace("'", "")  # Remove unwanted single quotes if they exist
    text = text.replace('"', '').strip()
    text = text.strip()  # Ensure no leading/trailing spaces

    return text
# Load dataset
df = pd.read_csv("augmented_dataset.csv")

# Apply paraphrasing to the 'prompt' column
df["prompt"] = df["prompt"].apply(lambda x: paraphrase(x))
df = df[df["prompt"].str.strip() != ""]

# Save the augmented dataset
df.to_csv("paraphrased_augmented_dataset.csv", index=False)
print("Paraphrased augmented dataset saved as paraphrased_augmented_dataset.csv.")






How do i prepare a budget
What is the best manner to plan my monthly spending
Can you assist me with this at my expense
Aid me with my disbursement
How much of my income should i write
How much should i pull through
How much of my income should i save for retirement
What portion of your income should go to rescue
How much of income should go to nest egg
Is there a word for this linguistic rule of thumb
Is there a savings rule, or is there a rescue formula
How much should i save for retreat
Of my own volition, I should kick the bucket to my retirement investment company
What is the best fashion to save for retreat
How do i salve for retreat
What are you going to tax write - sour
What are tax indite - and how do you pay off for income tax
Can you explicate revenue enhancement credits
Please excuse tax credits
Can you excuse the lack of recognition
How can i trim down my nonexempt income
What is the difference between condition and life indemnity
How do i choose the right health indemnity

In [ ]:
synonym_df = pd.read_csv("synonym_augmented_dataset.csv")
back_translation_df = pd.read_csv("back_translation_augmented_dataset.csv")
paraphrased_df = pd.read_csv("paraphrased_augmented_dataset.csv")

# Combine datasets
combined_df = pd.concat([synonym_df, back_translation_df, paraphrased_df], ignore_index=True)
combined_df.to_csv("combined_augmented_dataset.csv", index=False)
print("Combined augmented dataset saved as combined_augmented_dataset.csv.")


Combined augmented dataset saved as combined_augmented_dataset.csv.


In [ ]:
import pandas as pd

# Load the dataset
csv_file_path = "augmented_dataset.csv"
print(f"Loading dataset from {csv_file_path}.")
augmented_train_df = pd.read_csv(csv_file_path)

# Clean the 'prompt' column by removing the brackets and ensuring it's a single string
augmented_train_df["prompt"] = augmented_train_df["prompt"].apply(lambda x: ''.join(eval(x)).strip("[]").replace("'", "").strip() if isinstance(eval(x), list) else x)

# Print the cleaned DataFrame
print(augmented_train_df)

# # Save the cleaned dataset
# augmented_train_df.to_csv("cleaned_augmented_dataset.csv", index=False)


Loading dataset from augmented_dataset.csv.
                                               prompt  \
0                           How do i prepare a budget   
1   What endorse the best manner to plan my monthl...   
2               Can you assist me organize my expense   
3                         Aid me form my disbursement   
4                How much of my income should i write   
5                      How much should i pull through   
6   How much of my income should i save for retire...   
7          What portion of income should go to rescue   
8            How much of income should go to nest egg   
9        Is there a delivery linguistic rule of thumb   
10  Is there a savings rule is there a rescue formula   
11                 How much should i save for retreat   
12  Of my income should kick the bucket to my reti...   
13   What second the best fashion to save for retreat   
14                         How do i salve for retreat   
15                 What are taxation tax wri

In [ ]:
combined_df = pd.concat([augmented_train_df, combined_df], ignore_index=True)
combined_df.to_csv("cleaned_augmented_dataset.csv", index=False)
print("Combined augmented dataset saved as cleaned_augmented_dataset.csv.")

Combined augmented dataset saved as cleaned_augmented_dataset.csv.


In [ ]:
from sklearn.model_selection import train_test_split
# Load augmented_finance_dataset
csv_file_path = "cleaned_augmented_dataset.csv"
print(f"Loading dataset from {csv_file_path}.")
augmented_train_df = pd.read_csv(csv_file_path)

# Assume 'augmented_train_df' is your DataFrame
train_df, val_df = train_test_split(augmented_train_df, test_size=0.2, stratify=augmented_train_df['response'])

print(f"Train set size: {len(train_df)}")
print(f"Validation set size: {len(val_df)}")
# Print the class distribution in the train and validation sets to verify consistency
print("Train set class distribution:")
print(train_df['response'].value_counts())

print("Validation set class distribution:")
print(val_df['response'].value_counts())

Loading dataset from cleaned_augmented_dataset.csv.
Train set size: 92
Validation set size: 24
Train set class distribution:
response
Aim to save at least 20 of your income but adjust based on your financial goals                                                                             22
Tax deductions lower your taxable income while tax credits reduce your tax bill directly keep track of eligible expenses to maximize savings                19
It's recommended to save at least 15 of your income for retirement consider using taxadvantaged accounts like 401k or ira                                   13
Start by listing your income subtracting fixed expenses and allocating funds to savings and discretionary spending                                          13
Term life insurance covers you for a fixed period while whole life insurance includes a cash value component choose based on your needs and budget          10
Track your spending to identify nonessential expenses set a realistic s

In [ ]:
# Preprocess datasets
def preprocess_function(examples):
    inputs = tokenizer(
        examples["prompt"],
        max_length=128,
        truncation=True,
        padding="max_length"
    )
    labels = tokenizer(
        examples["response"],
        max_length=128,
        truncation=True,
        padding="max_length"
    )["input_ids"]
    inputs["labels"] = labels
    return inputs

# Convert pandas DataFrames to Hugging Face Dataset objects
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)
print(train_dataset)

# Before passing to Trainer, ensure that only necessary columns are present
train_dataset = train_dataset.remove_columns(['__index_level_0__'])
val_dataset = val_dataset.remove_columns(['__index_level_0__'])

print(train_dataset)

Map:   0%|          | 0/92 [00:00<?, ? examples/s]

Map:   0%|          | 0/24 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt', 'response', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 92
})
Dataset({
    features: ['prompt', 'response', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 92
})


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=7c97244e74deb1a68b75968e3546b0241d66926fe4d2d759849b6d0b530b9bb8
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
# !pip install wandb
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
from transformers import EarlyStoppingCallback
from nltk.translate.meteor_score import meteor_score
from bert_score import score
import os
os.environ["WANDB_MODE"] = "disabled"


# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    learning_rate=3e-5,
    save_total_limit=2,
    save_steps=500,
    eval_strategy="steps",
    eval_steps=100,
    logging_steps=50,
    logging_dir="./logs",
    load_best_model_at_end=True,
    remove_unused_columns=False,
    report_to=None,  # Disable W&B logging
    fp16=True,  # Enable mixed precision for faster training (if GPU supports it)
)

bleu_metric = load("bleu")
rouge_metric = load("rouge")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # BLEU Score
    bleu = bleu_metric.compute(predictions=decoded_preds, references=[[ref] for ref in decoded_labels])

    # ROUGE Score
    rouge = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels)

    # METEOR Score
    meteor = meteor_score(decoded_labels, decoded_preds)

    # BERTScore
    P, R, F1 = score(decoded_preds, decoded_labels, lang="en")

    return {
        "bleu": bleu["bleu"],
        "rougeL": rouge["rougeL"],
        "meteor": meteor,
        "bert_precision": P.mean().item(),
        "bert_recall": R.mean().item(),
        "bert_f1": F1.mean().item()
    }
early_stopping = EarlyStoppingCallback(early_stopping_patience=3)
# Fine-tune the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,  # Added metrics
    callbacks=[early_stopping]  # Adding early stopping callback
)
trainer.train()

# Save the fine-tuned model
trainer.save_model("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")
print("Fine-tuned model saved successfully.")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Fine-tuned model saved successfully.


In [ ]:
from evaluate import load
import torch

# Load BLEU and ROUGE metrics
bleu_metric = load("bleu")
rouge_metric = load("rouge")

# Evaluation function
def evaluate_model(dataset):
    preds, refs = [], []
    for example in dataset:
        # Tokenize the prompt
        inputs = tokenizer(
            example["prompt"],
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=128,
        )
        # Move inputs to the same device as the model
        inputs = {key: value.to(device) for key, value in inputs.items()}

        # Generate the response
        outputs = model.generate(
            inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=100,  # Increased max length
            num_beams=5,     # Beam search for diversity
            early_stopping=True,
            pad_token_id=tokenizer.eos_token_id,
        )
        # Decode the generated response
        pred = tokenizer.decode(outputs[0], skip_special_tokens=True)
        preds.append(pred)
        refs.append(example["response"])

    # Compute BLEU score
    bleu_score = bleu_metric.compute(
        predictions=preds, references=[[r] for r in refs]
    )
    # Compute ROUGE score
    rouge_score = rouge_metric.compute(predictions=preds, references=refs)
    return bleu_score, rouge_score


# Ensure model is on the correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Ensure val_dataset is preprocessed and available
bleu_score, rouge_score = evaluate_model(val_dataset)

# Print the results
print(f"BLEU Score: {bleu_score}")
print(f"ROUGE Score: {rouge_score}")





BLEU Score: {'bleu': 0.5777146650460923, 'precisions': [0.6984126984126984, 0.6258992805755396, 0.5877862595419847, 0.5691056910569106], 'brevity_penalty': 0.9342350518693858, 'length_ratio': 0.9363057324840764, 'translation_length': 441, 'reference_length': 471}
ROUGE Score: {'rouge1': 0.6734944562701444, 'rouge2': 0.6071989201101043, 'rougeL': 0.6508059590252186, 'rougeLsum': 0.6499702727024472}


In [33]:
import torch

# Function to generate a response
def generate_response(prompt, model, tokenizer, device):
    # Tokenize the input prompt
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128  # Adjust as needed
    ).to(device)  # Ensure tensors are on the same device as the model

    # Generate the response
    outputs = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_length=100,  # Maximum length of the generated response
        num_beams=5,     # Beam search for diversity
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode the generated tokens to text
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Example usage
prompt = "What are the main components of a financial trading system?"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)  # Ensure model is on the correct device

response = generate_response(prompt, model, tokenizer, device)
print(f"Generated Response: {response}")




Generated Response: It's recommended to save at least 20 of your income but adjust based on your financial goals
